In [27]:
import string

In [28]:
queries = [
    'Did I maintain a speed of 20 mph for 5 minutes after biking an incline of 5 degrees?'
]

In [102]:
triggerPhrases = [
    ['did i', 'do i'],
    ['how many', 'how much']
]
# Should contain all recognizable words
synonyms = [
    ['maintain'],
    ['after'],
    ['until'],
    ['greater'],
    ['less'],
    ['speed', 'velocity'],
    ['distance'],
    ['incline', 'hill'],
    ['altitude', 'elevation', 'height'],
    ['degree', 'º', '°', 'deg'],
    ['mph', 'miles per hour'],
    ['second'],
    ['minute'],
    ['hour'],
    ['meter'],
    ['foot'],
    ['of']
]
unitWords = [  # Omit s if can be plural
    'degree', 'mph', 'second', 'minute', 'hour', 'meter', 'foot'
]
connectingWords = [
    'of'
]
operatorWords = [
    'maintain', 'after', 'until', 'greater', 'less'
]
variableWords = [
    'speed', 'altitude', 'distance', 'incline'
]
operatorForms = {
    'maintain': [
        ['maintain', 'BOOL_EXPR', 'VALUE']
        ###['maintain', 'BOOL_EXPR', 'until']
    ],
    'after': [
        ['BOOL_EXPR', 'after', 'BOOL_EXPR']
    ],
    #'until': []
    'greater': [
        ['REAL_EXPR', 'greater', 'REAL_EXPR']
    ],
    'less': [
        ['REAL_EXPR', 'less', 'REAL_EXPR']
    ]
    # ...
}

In [103]:
synonymDict = {}
for group in synonyms:
    standardWord = group[0]
    for word in group:
        synonymDict[word] = standardWord

In [86]:
all = string.maketrans('','')
noDigs = all.translate(all, string.digits)
noLetters = all.translate(all, string.letters)
allowedLetters = ' abcdefghijklmnopqrstuvwxyz0123456789%.º°'
notAllowed = all.translate(all, allowedLetters)
def cleanQuery(query):
    return query.lower().translate(all, notAllowed)

In [87]:
def wordToStandard(word):
    if word is None:
        return None
    word = word.lower()  # Lower case
    if len(word) == 0:
        return None
    if word[0] in '-0123456789':
        #TODO: Maybe more processing of numbers
        return word
    if word in synonymDict:
        return synonymDict[word]
    word = word.translate(all, noLetters)  # Keep only letters
    if word in synonymDict:
        return synonymDict[word]
    if word[-1] == 's':
        word = word[:-1]
        if word in synonymDict:
            return synonymDict[word]
    return None

In [107]:
class Token:
    '''A word associated with its meaning'''
    # Token types
    UNKNOWN = 0
    NONE = 1
    OPERATOR = 2
    CONNECTOR = 3
    NUMBER = 4
    UNIT = 5
    VARIABLE = 6
    def __init__(self, word, left=None, right=None):
        self.origWord = word
        self.word = wordToStandard(word)
        self.left = wordToStandard(left)
        self.right = wordToStandard(right)
        if self.word in operatorWords:
            self.type = Token.OPERATOR
        elif self.word in variableWords:
            self.type = Token.VARIABLE
        elif self.word in unitWords:
            self.type = Token.UNIT
        elif self.word in connectingWords:
            self.type = Token.CONNECTOR
        elif len(self.word) > 0 and self.word[0] in '-0123456789':
            self.type = Token.NUMBER
        else:
            self.type = NONE
            if self.word is not None:
                print repr(self.origWord), repr(self.word), 'not understood.'
    def __str__(self):
        typeStr = ['UNKNOWN', 'NONE', 'OPERATOR', 'CONNECTOR',
                   'NUMBER', 'UNIT', 'VARIABLE'][self.type]
        return '<%s, %s>' % (self.word, typeStr)
    def __repr__(self):
        return self.__str__()

In [108]:
class TokenGroup:
    '''A group of tokens that mean somthing as a group'''
    # Group types
    UNKOWN = 100
    VALUE = 101  # Number with unit
    BOOL_EXPR = 102  # Logical or temporal
    REAL_EXPR = 103
    tokenPatterns = {
        VALUE: [
            [Token.NUMBER, Token.UNIT],
            [Token.NUMBER]
        ],
        BOOL_EXPR: [
            # Equality operator
            [Token.VARIABLE, Token.CONNECTOR, VALUE],
            [VALUE, Token.VARIABLE],
            []
        ]
    }
    def __init__(self, type, tokens):
        self.type = type
        self.tokens = tokens
    def __str__(self):
        typeStr = ['UNKNOWN', 'VAL', 'EXPRESSION'][self.type-100]
        if self.type == TokenGroup.UNKNOWN:
            return '<TokenGroup UNKNOWN>'
        if self.type == TokenGroup.VALUE:
            return '<VAL %s %s>' 
        if self.type == TokenGroup.BOOL_EXPR:
            return '<EXPR>'
        if self.type == TokenGroup.REAL_EXPR:
            return '<EXPR>'
            #return '<%s, %s>' % (repr(self.word), typeStr)
    def __repr__(self):
        return self.__str__()

In [109]:
def splitQuery(query):
    query = cleanQuery(query)
    l = query.split()
    tokens = []
    for word in l:
        standard = wordToStandard(word)
        if standard is not None:
            tokens.append(standard)
    return tokens

In [110]:
splitQuery(queries[0])

['maintain',
 'speed',
 'of',
 '20',
 'mph',
 '5',
 'minute',
 'after',
 'incline',
 'of',
 '5',
 'degree']

In [111]:
def tokenizeQuery(query):
    split = splitQuery(query)
    tokens = []
    for i in xrange(len(split)):
        left = None if i-1 < 0 else split[i-1]
        right = None if i+1 >= len(split) else split[i+1]
        t = Token(split[i], left, right)
        if t.type != Token.UNKNOWN and t.type != Token.NONE:
            tokens.append(t)
    return tokens

In [112]:
tokenizeQuery(queries[0])

[<maintain, OPERATOR>,
 <speed, VARIABLE>,
 <of, CONNECTOR>,
 <20, NUMBER>,
 <mph, UNIT>,
 <5, NUMBER>,
 <minute, UNIT>,
 <after, OPERATOR>,
 <incline, VARIABLE>,
 <of, CONNECTOR>,
 <5, NUMBER>,
 <degree, UNIT>]

In [80]:
Token('until')

<None, UNIT>